## Train a character-level GPT on some text data

The inputs here are simple text files, which we chop up to individual characters and then train GPT on. So you could say this is a char-transformer instead of a char-rnn. Doesn't quite roll off the tongue as well. In this example we will feed it some Shakespeare, which we'll get it to predict character-level.

In [16]:
# set up logging
%load_ext autoreload
%autoreload 2
import logging
# logging.basicConfig(
#         format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
#         datefmt="%m/%d/%Y %H:%M:%S",
#         level=logging.INFO,
# )
logging.getLogger().setLevel(logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [18]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [19]:
%%writefile dataset.py

import math
import torch
from torch.utils.data import Dataset

class CharDataset(Dataset):

    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.block_size + 1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y


Overwriting dataset.py


In [20]:
block_size = 128 # spatial extent of the model for its context

In [21]:
!curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -o input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  6367k      0 --:--:-- --:--:-- --:--:-- 6369k


In [22]:
# you can download this file at https://github.com/karpathy/char-rnn/blob/master/data/tinyshakespeare/input.txt
from dataset import CharDataset
text = open('input.txt', 'r').read() # don't worry we won't run out of file handles
train_dataset = CharDataset(text[:1000], block_size) # one line of poem is roughly 50 characters
print(train_dataset.stoi)
print(train_dataset.itos)

data has 1000 characters, 46 unique.
{'\n': 0, ' ': 1, '!': 2, "'": 3, ',': 4, '.': 5, ':': 6, ';': 7, '?': 8, 'A': 9, 'B': 10, 'C': 11, 'F': 12, 'I': 13, 'L': 14, 'M': 15, 'N': 16, 'O': 17, 'R': 18, 'S': 19, 'W': 20, 'Y': 21, 'a': 22, 'b': 23, 'c': 24, 'd': 25, 'e': 26, 'f': 27, 'g': 28, 'h': 29, 'i': 30, 'j': 31, 'k': 32, 'l': 33, 'm': 34, 'n': 35, 'o': 36, 'p': 37, 'r': 38, 's': 39, 't': 40, 'u': 41, 'v': 42, 'w': 43, 'y': 44, 'z': 45}
{0: '\n', 1: ' ', 2: '!', 3: "'", 4: ',', 5: '.', 6: ':', 7: ';', 8: '?', 9: 'A', 10: 'B', 11: 'C', 12: 'F', 13: 'I', 14: 'L', 15: 'M', 16: 'N', 17: 'O', 18: 'R', 19: 'S', 20: 'W', 21: 'Y', 22: 'a', 23: 'b', 24: 'c', 25: 'd', 26: 'e', 27: 'f', 28: 'g', 29: 'h', 30: 'i', 31: 'j', 32: 'k', 33: 'l', 34: 'm', 35: 'n', 36: 'o', 37: 'p', 38: 'r', 39: 's', 40: 't', 41: 'u', 42: 'v', 43: 'w', 44: 'y', 45: 'z'}


In [29]:
from mingpt.model import GPT, GPTConfig
from torch.utils.data.dataloader import DataLoader
import logging
logging.getLogger().setLevel(logging.INFO)

mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=8, n_head=8, n_embd=512)
model = GPT(mconf)

## _----

loader = DataLoader(
    train_dataset,
    shuffle=True,
    pin_memory=True,
    batch_size=1,
    num_workers=0,
)

for x,y in loader:
    model(x,y)
    break

idx tensor([[30, 40, 30, 45, 26, 35,  6,  0, 20, 26,  1, 22, 38, 26,  1, 22, 24, 24,
         36, 41, 35, 40, 26, 25,  1, 37, 36, 36, 38,  1, 24, 30, 40, 30, 45, 26,
         35, 39,  4,  1, 40, 29, 26,  1, 37, 22, 40, 38, 30, 24, 30, 22, 35, 39,
          1, 28, 36, 36, 25,  5,  0, 20, 29, 22, 40,  1, 22, 41, 40, 29, 36, 38,
         30, 40, 44,  1, 39, 41, 38, 27, 26, 30, 40, 39,  1, 36, 35,  1, 43, 36,
         41, 33, 25,  1, 38, 26, 33, 30, 26, 42, 26,  1, 41, 39,  6,  1, 30, 27,
          1, 40, 29, 26, 44,  0, 43, 36, 41, 33, 25,  1, 44, 30, 26, 33, 25,  1,
         41, 39]]) torch.Size([1, 128])
targets tensor([[40, 30, 45, 26, 35,  6,  0, 20, 26,  1, 22, 38, 26,  1, 22, 24, 24, 36,
         41, 35, 40, 26, 25,  1, 37, 36, 36, 38,  1, 24, 30, 40, 30, 45, 26, 35,
         39,  4,  1, 40, 29, 26,  1, 37, 22, 40, 38, 30, 24, 30, 22, 35, 39,  1,
         28, 36, 36, 25,  5,  0, 20, 29, 22, 40,  1, 22, 41, 40, 29, 36, 38, 30,
         40, 44,  1, 39, 41, 38, 27, 26, 30, 40, 39,  1, 

In [25]:
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(
    max_epochs=2,
    batch_size=512,
    learning_rate=6e-4,
    lr_decay=True,
    warmup_tokens=512 * 20,
    final_tokens=2 * len(train_dataset) * block_size,
    num_workers=0,
)
trainer = Trainer(model, train_dataset, None, tconf)
trainer.train()

  0%|          | 0/2 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [10]:
# alright, let's sample some character-level Shakespeare
from mingpt.utils import sample

context = "O God, O God!"
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
y = sample(model, x, 2000, temperature=1.0, sample=True, top_k=10)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print(completion)

O God, O God! that e'er this tongue of mine,
That laid the sentence of dread banishment
On yon proud man, should take it off again
With words of sooth! O that I were as great
As is my grief, or lesser than my name!
Or that I could forget what I have been,
Or not remember what I must be now!
Swell'st thou, proud heart? I'll give thee scope to beat,
Since foes have scope to beat both thee and me.

DUKE OF AUMERLE:
Northumberland comes back from Bolingbroke.

KING RICHARD II:
What must the king do now? must he submit?
The king shall do it: must he be deposed?
The king shall be contented: must he lose
The name of king? o' God's name, let it go:
I'll give my jewels for a set of beads,
My gorgeous palace for a hermitage,
My gay apparel for an almsman's gown,
My figured goblets for a dish of wood,
My sceptre for a palmer's walking staff,
My subjects for a pair of carved saints
And my large kingdom for a little grave,
A little little grave, an obscure grave;
Or I'll be buried in the king's hig

In [11]:
# well that was fun